<a href="https://colab.research.google.com/github/quanvu0996/compet/blob/master/Kaggle_compet/WiDS_2021/wids_quanvh8_run_for_stacking_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ENVIROMENT SETTING

In [1]:
# COLAB ONLY
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install octopus-ml
!pip install catboost
!pip install bayesian-optimization
!pip install missingpy 

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import octopus_ml as oc
import sklearn as sk

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn import preprocessing
from sklearn import ensemble, naive_bayes, svm, linear_model
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,roc_curve, auc, confusion_matrix
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA, TruncatedSVD

from multiprocessing.pool import ThreadPool
from multiprocessing import Pool 
from scipy.stats import ks_2samp

from xgboost import XGBClassifier
from catboost import Pool, cv, CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier
import lightgbm as lgb
import xgboost as xgb

# from pytorch_tabnet.tab_model import TabNetClassifier
import torch

import glob

from bayes_opt import BayesianOptimization
import missingpy 
import warnings
warnings.filterwarnings("ignore")

# COMMON FUNCTION

In [4]:
# Function lưu kết quả thành 3 file cho stacking
# submit thành 3 file: train, valid, test prediction; y_pred của valid và y_true_valid sẽ được dùng để train 
def _to_save_model_predict_3files(model, file_name, _X_train, _X_valid, _X_test):
    X_train['prediction'] = model.predict(_X_train)
    X_train['diabetes_mellitus'] = y_train

    X_valid['prediction'] = model.predict(_X_valid)
    X_valid['diabetes_mellitus'] = y_valid

    df_test['diabetes_mellitus'] = model.predict( _X_test )

    # save to csv
    X_train[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_train/train_'+file_name+'.csv', index = True)
    X_valid[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_valid/valid_'+file_name+'.csv', index = True)
    df_test[ 'diabetes_mellitus'].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_test/test_'+file_name+'.csv', index = True)
    print('results saved')

In [5]:
# Convert data 64bit to 32bit if possible to reduce memory
def reduce_mem_usage(df: pd.DataFrame,
                     verbose: bool = True) -> pd.DataFrame:
    
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2

    for col in df.columns:
        col_type = df[col].dtypes

        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            if str(col_type)[:3] == 'int':

                if (c_min > np.iinfo(np.int32).min
                      and c_max < np.iinfo(np.int32).max):
                    df[col] = df[col].astype(np.int32)
                elif (c_min > np.iinfo(np.int64).min
                      and c_max < np.iinfo(np.int64).max):
                    df[col] = df[col].astype(np.int64)
            else:
                if (c_min > np.finfo(np.float16).min
                        and c_max < np.finfo(np.float16).max):
                    df[col] = df[col].astype(np.float16)
                elif (c_min > np.finfo(np.float32).min
                      and c_max < np.finfo(np.float32).max):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    reduction = (start_mem - end_mem) / start_mem

    msg = f'Mem. usage decreased to {end_mem:5.2f} MB ({reduction * 100:.1f} % reduction)'
    if verbose:
        print(msg)

    return df

# BỘ DATA V5 VỚI MISSING VALUES ĐOÁN BẰNG MODEL

---



In [6]:
folder_path = '/content/drive/MyDrive/Data/colabs_data/WiDS'
df_full = reduce_mem_usage(
            pd.read_csv(folder_path + "/WiDS2021/Wids2021_full_feature_v5_clean_missing_values.csv") )
df_full.set_index('encounter_id', inplace = True)

Mem. usage decreased to 482.80 MB (67.7 % reduction)


In [7]:
# Convert boolean columns
cols_boolean = df_full.select_dtypes('bool').columns
df_full[cols_boolean] = df_full[cols_boolean].astype('int8')
df_full.replace([np.inf, -np.inf], 0, inplace = True)

In [8]:
# Data type 
col_target = 'diabetes_mellitus'
cols_id = ['hospital_id','encounter_id', 'Unnamed: 0', 'Unnamed: 0.1','knn_value_16', 'knn_value_512', 'knn_value_64' ]# , 'total_chronic', 'has_complicator', 'has_history', 'apache_hist'
cols_cat = [i for i in df_full.select_dtypes('object').columns if i not in cols_id + [col_target]]
cols_num = [i for i in df_full.columns if i not in cols_cat + cols_id + [col_target] ]
cols_ft = cols_cat + cols_num

In [9]:
# Split data set
df_train, df_test = df_full[:130157], df_full[130157:]
X_train, X_valid, y_train, y_valid = train_test_split( 
    df_train[cols_ft], df_train[col_target], train_size= 98000, random_state= 68)

### Check dữ liệu bộ v5 with missing filled

In [ ]:
print 1

In [10]:
df_full['apache_2_diagnosis'].unique()

array([113. , 108. , 122. , 203. , 119. , 301. , 116. , 112. , 303. ,
       218. , 304. , 302. , 305. , 124. , 202. , 207. , 110. , 209. ,
       109. , 106. , 117. , 120. , 193.9, 217. , 114. , 102. , 308. ,
       105. , 212. , 219. , 306. , 121. , 214. , 123. , 213. , 208. ,
       101. , 118. , 307. , 215. , 192.6, 192.4, 103. , 192.2, 115. ,
       194.8, 104. , 216. , 107. , 193.6, 192.8, 192.5, 193.5, 194.9,
       193.8, 190. , 189.9], dtype=float16)

In [11]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 140391 entries, 214826 to 136852
Columns: 1393 entries, Unnamed: 0 to knn_value_512
dtypes: float16(999), float32(47), int32(340), int64(1), object(6)
memory usage: 483.3+ MB


In [12]:
df_full['encounter_id']

KeyError: ignored

In [ ]:
x = df_full.isnull().sum()
cols_miss = list(x[x>0].index )
df_full[cols_miss ].info()

In [ ]:
# df_full[cols_cat] =  df_full[cols_cat].fillna('MISSING')
df_full[cols_num] = df_full[cols_num].fillna(df_full[cols_num].mean())

In [ ]:
# df_full[ df_full['bun_max_class_mean_glucose'].isnull()]
df_full['bun_max_class_mean_glucose'].describe()


In [ ]:
ms_pipe = SimpleImputer(strategy='median')
x = ms_pipe.fit_transform(df_full[cols_miss])

In [ ]:
x

In [ ]:
# df_full[cols_miss] = x
df_full[cols_miss]

In [ ]:
df_full.to_csv(folder_path + "/WiDS2021/Wids2021_full_feature_v5_clean_missing_values.csv")

### Model 1. LightGBM

In [21]:
X_train

,ethnicity,gender,hospital_admit_source,icu_admit_source,icu_stay_type,icu_type,age,bmi,elective_surgery,height,icu_id,pre_icu_los_days,readmission_status,weight,albumin_apache,apache_2_diagnosis,apache_3j_diagnosis,apache_post_operative,arf_apache,bilirubin_apache,bun_apache,creatinine_apache,fio2_apache,gcs_eyes_apache,gcs_motor_apache,gcs_unable_apache,gcs_verbal_apache,glucose_apache,heart_rate_apache,hematocrit_apache,intubated_apache,map_apache,paco2_apache,paco2_for_ph_apache,pao2fio2ratio_apache,arterial_ph_apache,resprate_apache,sodium_apache,temp_apache,urineoutput_apache,...,mbp_sum_na_full_invasive,sysbp_sum_na_invasive,sysbp_sum_na_noninvasive,sysbp_sum_na_full_invasive,NA_info,glucose_na_h,glucose_na_d,glucose_na,bun_na_h,bun_na_d,bun_na,creatinine_na_h,creatinine_na_d,creatinine_na,gender_mean_glucose,gender_std_glucose,age_mean_glucose,age_std_glucose,ageclass_mean_glucose,ageclass_std_glucose,weightclass_mean_glucose,weightclass_std_glucose,bun_max_class_mean_glucose,bun_max_class_std_glucose,d1_creatinine_max_bins_mean_glucose,d1_creatinine_max_bins_std_glucose,ethnicity_mean_glucose,ethnicity_std_glucose,hospital_admit_source_mean_glucose,hospital_admit_source_std_glucose,icu_admit_source_mean_glucose,icu_admit_source_std_glucose,icu_stay_type_mean_glucose,icu_stay_type_std_glucose,icu_type_mean_glucose,icu_type_std_glucose,apache_2_diagnosis_mean_glucose,apache_2_diagnosis_std_glucose,int_apache_3j_diag_mean_glucose,int_apache_3j_diag_std_glucose
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
249554,Caucasian,M,Emergency Department,Accident & Emergency,admit,Cardiac ICU,66.00000,34.937500,0.0,182.875,0.0,0.551758,0,116.87500,2.900391,305.0,901.0,0,0.0,0.300049,76.000000,4.679688,0.500000,3.0,5.0,0.0,1.0,388.000,99.0,17.796875,0.0,45.0,32.00000,32.00000,170.0000,7.390625,31.0,132.000,37.09375,8720.0,...,4,4,0,4,316,0,0,0,2,2,4,2,2,4,173.25,85.0000,182.875,86.5000,179.375,82.2500,180.125,85.875,207.125,113.8125,194.125,106.0000,172.25,83.7500,176.500,97.9375,176.375,96.625,173.875,86.1875,176.000,91.6250,167.625,78.93750,172.625,85.18750
156140,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,52.15625,21.203125,0.0,147.250,0.0,0.513184,0,46.00000,2.900391,303.0,211.0,0,0.0,0.600098,45.000000,0.700195,0.556641,4.0,6.0,0.0,4.0,112.000,109.0,29.000000,0.0,56.0,39.09375,39.09375,106.0000,7.312500,36.0,137.000,37.09375,1597.0,...,4,4,0,4,416,2,0,2,2,2,4,2,2,4,174.75,87.6875,177.250,86.5000,176.625,89.3125,164.625,84.625,189.500,96.1250,166.625,77.2500,172.25,83.7500,176.500,97.9375,176.375,96.625,173.875,86.1875,176.125,90.7500,170.875,71.06250,169.750,74.75000
243930,Caucasian,M,MISSING,Accident & Emergency,admit,MICU,75.00000,33.218750,0.0,177.750,0.0,0.522461,0,105.00000,3.699219,112.0,107.0,0,0.0,1.000000,14.000000,1.049805,0.555664,4.0,6.0,0.0,5.0,104.000,40.0,41.312500,0.0,57.0,39.15625,39.15625,106.0000,7.312500,45.0,137.000,35.59375,1855.0,...,2,2,0,2,360,2,0,2,2,2,4,2,2,4,173.25,85.0000,177.375,79.8125,170.500,75.5000,180.125,85.875,158.125,69.8125,166.625,77.2500,172.25,83.7500,174.625,85.1250,176.375,96.625,173.875,86.1875,179.125,95.9375,169.250,84.62500,169.250,84.62500
216008,Caucasian,F,Emergency Department,Accident & Emergency,admit,Med-Surg ICU,57.00000,40.781250,0.0,170.000,0.0,0.542480,0,117.87500,2.933594,301.0,410.0,0,0.0,0.952148,21.265625,1.241211,0.554199,2.0,4.0,0.0,1.0,129.375,101.0,34.781250,0.0,56.0,39.37500,39.37500,106.0000,7.308594,18.0,138.375,36.31250,1248.0,...,4,4,0,4,613,2,2,4,2,2,4,2,2,4,174.75,87.6875,180.000,93.7500,179.750,89.3125,190.750,90.750,158.125,69.8125,166.625,77.2500,172.25,83.7500,176.500,97.9375,176.375,96.625,173.875,86.1875,176.125,90.7500,157.000,70.87500,158.625,77.75000
269221,Caucasian,F,MISSING,Floor,admit,Neuro ICU,61.00000,38.687500,0.0,170.000,0.0,0.926758,0,111.81250,2.906250,102.0,206.0,0,0.0,0.916992,34.000000,1.099609,0.592773,3.0,6.0,0.0,1.0,112.000,111.0,35.906250,0.0,60.0,46.3

In [20]:
to_train_data = lgb.Dataset(X_train[cols_num], y_train)
to_val_data = lgb.Dataset(X_valid[cols_num], y_valid)

#### Model 1.1. lgbm tham so 1

In [ ]:
# Without categorical columns| 20210225
best_params =  {'bagging_fraction': 0.8766805307530916,
  'feature_fraction': 0.8642027679654405,
  'lambda_l1': 4.912602026444597,
  'lambda_l2': 2.437499183488889,
  'learning_rate': 0.02501159905983583,
  'min_child_weight': 10.077249918461563,
  'min_split_gain': 0.05624653693483012,
  'num_leaves': 32.35902690851246,
  'scale_pos_weight': 0.7903383094151535}
  
def lgb_params( params ):
    params2 = {'boosting_type': 'gbdt', 'device' : 'cpu','application':'binary', 'early_stopping_round':400, 
               'metric':'auc', 'max_depth':-1, 'force_col_wise': True}
    params['feature_fraction'] = max(min(params['feature_fraction'], 1), 0)
    params['bagging_fraction'] = max(min(params['bagging_fraction'], 1), 0)
    params['num_leaves'] = int(round(params['num_leaves']))
    params['lambda_l1'] = max(params['lambda_l1'], 0)
    params['learning_rate'] = max(params['learning_rate'], 0)
    params['lambda_l2'] = max(params['lambda_l2'], 0)
    return dict( params, **params2)

model = lgb.train( lgb_params(best_params) , to_train_data, verbose_eval =200, 
                    valid_sets = to_val_data, num_boost_round = 5500 )

#### Model 1.2 . lgbm tham so 2

In [19]:
# Without categorical columns| 20210225
best_params =  {'bagging_fraction': 1,
  'feature_fraction': 0.25,
  'lambda_l1': 3,
  'lambda_l2': 1,
  'learning_rate': 0.007,
  'min_child_weight': 10.077249918461563,
  'min_split_gain': 0.05624653693483012,
  'num_leaves': 32.35902690851246,
  'scale_pos_weight': 3}
  
def lgb_params( params ):
    params2 = {'boosting_type': 'gbdt', 'device' : 'cpu','application':'binary', 'early_stopping_round':400, 
               'metric':'auc', 'max_depth':-1, 'force_col_wise': True}
    params['feature_fraction'] = max(min(params['feature_fraction'], 1), 0)
    params['bagging_fraction'] = max(min(params['bagging_fraction'], 1), 0)
    params['num_leaves'] = int(round(params['num_leaves']))
    params['lambda_l1'] = max(params['lambda_l1'], 0)
    params['learning_rate'] = max(params['learning_rate'], 0)
    params['lambda_l2'] = max(params['lambda_l2'], 0)
    return dict( params, **params2)

model = lgb.train( lgb_params(best_params) , to_train_data, verbose_eval =200, 
                    valid_sets = to_val_data, num_boost_round = 5500 )

NameError: ignored

In [ ]:
# dữ liệu đã có encounter_id là index
_to_save_model_predict_3files(model, 'lgbm_ftv5m_gbdt_2', 
                              X_train[cols_num], X_valid[cols_num], df_test[cols_num])

### Model 2. XGBoost

In [18]:
def get_params( max_depth, gamma, min_child_weight, max_delta_step, subsample, colsample_bytree ):
    params = {# fixed params
              'booster' : 'gbtree', 'seed' : 1001,
              'eta' : 0.1,
              'objective' : 'binary:logistic',
              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'auc',
              # 'gpu_id' : 0,
              # 'tree_method' : 'gpu_hist',
              # to search params
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              }
    return params

In [ ]:
best_search = {'colsample_bytree': 0.7888830791688737,
 'gamma': 4.7424296800429175,
 'max_delta_step': 9.498869873848605,
 'max_depth': 4.508336667348262,
 'min_child_weight': 9.56973791182546,
 'subsample': 0.7516726936641032}

to_train_data = xgb.DMatrix(X_train[cols_num], y_train)
to_val_data = xgb.DMatrix(X_valid[cols_num], y_valid)
to_test_data = xgb.DMatrix(df_test[cols_num])

In [ ]:
model = xgb.train( get_params(**best_search) , to_train_data, verbose_eval =200, evals = [(to_val_data, 'auc')],
                  num_boost_round = 20000, early_stopping_rounds = 500 )

In [ ]:
# dữ liệu đã có encounter_id là index
_to_save_model_predict_3files(model, 'xgb_ftv5m_gbdt', to_train_data, to_val_data, to_test_data )

### Model 3. ANN with numpy data

In [17]:
# MLP Nets
model = tf.keras.Sequential([
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation = 'selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(256, activation = 'selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation = 'selu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer = 'adam', loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = 0.015), 
              metrics = [tf.keras.metrics.AUC()])
history = model.fit( X_train[cols_num], y_train, 
    epochs=15,batch_size=1024,
    validation_data = ( X_valid[cols_num], y_valid), 
    # callbacks=[
    #     tf.keras.callbacks.ReduceLROnPlateau(
    #         monitor='val_loss', 
    #         factor=0.3, 
    #         patience=3,
    #         min_delta = 1e-3, 
    #         mode = 'min',
    #         verbose=1
    #     ),
    #     tf.keras.callbacks.EarlyStopping(
    #         monitor='val_loss',
    #         min_delta=0,
    #         patience=10,
    #         mode='auto',
    #         verbose=1,
    #         baseline=None,
    #         restore_best_weights=True
    #     )
    # ]
)

Epoch 1/15
96/96 [==============================] - ETA: 0s - loss: 0.6318 - auc_3: 0.7482

InvalidArgumentError: ignored

In [ ]:
# dữ liệu đã có encounter_id là index
_to_save_model_predict_3files(model, 'dl_mlp_ftv5m', X_train[cols_num], X_valid[cols_num], df_test[cols_num] )

### Model 4. 

# BỘ DATA 2020FEATURES KAGGLE

---




In [5]:
# Bộ feature 2020
folder_path = '/content/drive/MyDrive/Data/colabs_data/WiDS'
# derived features
df_train = pd.read_csv(folder_path + "/WiDS2021/train_df_2020features.csv")
df_test = pd.read_csv(folder_path + "/WiDS2021/test_df_2020features.csv")

# Đổi dạng bool thành int
cols_boolean = df_train.select_dtypes(include= 'boolean').columns
df_train[cols_boolean] = df_train[cols_boolean].astype('int')
df_test[cols_boolean] = df_test[cols_boolean].astype('int')
df_train.replace([np.inf, -np.inf], 0, inplace = True)
df_test.replace([np.inf, -np.inf], 0, inplace = True)

# Data type 
col_target = 'diabetes_mellitus'
cols_id = ['encounter_id', 'Unnamed: 0', 'Unnamed: 0.1','knn_value_16', 'knn_value_512', 'knn_value_64' ]
cols_cat = [i for i in df_train.select_dtypes('object').columns if i not in cols_id + [col_target]]
cols_num = [i for i in df_train if i not in cols_cat + cols_id + [col_target] ]
cols_ft = cols_cat + cols_num

df_train.set_index('encounter_id', inplace = True)
df_test.set_index('encounter_id', inplace = True)

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split( 
    df_train[cols_ft], df_train[col_target], train_size= 98000, random_state= 68)

In [13]:
X_train

,intubated_apache,glucose_wb,d1_arterial_po2_max,gcs_function,cat_bmi,d1_diasbp_invasive_max,h1_arterial_po2_min,h1_bilirubin_min,gcs_motor_apache,hepatic_failure,h1_creatinine_max,h1_calcium_max,h1_diasbp_invasive_min,h1_sodium_min,d1_arterial_po2_min,h1_hco3_min,d1_arterial_pco2_max,gcs_eyes_apache,hospital_admit_source,h1_bun_min,d1_bilirubin_min,icu_admit_source,d1_sysbp_noninvasive_min,h1_potassium_max,h1_sysbp_invasive_min,h1_hemaglobin_max,h1_arterial_ph_max,icu_stay_type,d1_lactate_min,h1_calcium_min,bilirubin_apache,cirrhosis,h1_diasbp_invasive_max,solid_tumor_with_metastasis,gender,d1_spo2_min,h1_potassium_min,gcs_verbal_apache,d1_diasbp_invasive_min,comorbidity_score,...,glucose_indicator,hematocrit_indicator,pao2fio2ratio_indicator,arterial_ph_indicator,resprate_indicator,sodium_indicator,temp_indicator,wbc_indicator,creatinine_indicator,total_day_more_extreme,d1_resprate_div_mbp_min,d1_resprate_div_sysbp_min,d1_lactate_min_div_diasbp_min,d1_heartrate_min_div_d1_sysbp_min,d1_hco3_div,d1_resprate_times_resprate,left_average_spo2,total_chronic,total_cancer_immuno,has_complicator,apache_3j,rank_frqenc_elective_surgery,rank_frqenc_icu_id,rank_frqenc_arf_apache,rank_frqenc_intubated_apache,rank_frqenc_ventilated_apache,rank_frqenc_cirrhosis,rank_frqenc_hepatic_failure,rank_frqenc_immunosuppression,rank_frqenc_solid_tumor_with_metastasis,rank_frqenc_apache_3j_diagnosis_x,rank_frqenc_apache_2_diagnosis_x,rank_frqenc_apache_3j,rank_frqenc_apache_3j_diagnosis_split1,rank_frqenc_apache_2_diagnosis_split1,rank_frqenc_gcs_sum_type,rank_frqenc_total_cancer_immuno,rank_frqenc_total_chronic,rank_frqenc_apache_2_diagnosis_type,rank_frqenc_apache_3j_diagnosis_type
encounter_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
249554,1,4,316.0,12,4,0.0,0.0,0.0,4,0,0.0,0.0,0.0,0.0,69.0,0.0,49.0,2,0,0.0,0.3,0,78.0,0.0,0.0,0.0,0.00,0,0.5,0.0,0.3,0,0.0,0,0,73.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,10,0.3777,0.21790,0.01428,0.6284,1.167,510.0,91.0,0,0,0,5,83574.5,41171.5,72116.5,11061.0,23186.0,71304.0,71132.5,71982.0,71643.5,57477.5,36594.0,3594.5,49912.5,7818.0,36690.5,73512.5,71757.5,83594.0,10676.5
156140,0,3,0.0,4,2,0.0,0.0,0.0,5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0,0.0,0.6,0,94.0,0.0,0.0,0.0,0.00,0,0.0,0.0,0.6,0,0.0,0,1,90.0,0.0,3,0.0,0,...,1,1,0,0,0,1,0,1,1,10,0.3215,0.19150,0.00000,0.9683,1.000,648.0,96.0,0,0,0,6,83574.5,8846.5,72116.5,81256.5,93381.5,71304.0,71132.5,71982.0,71643.5,72153.5,69659.5,67044.0,18443.5,124466.5,91465.0,73512.5,71757.5,83594.0,84044.0
243930,0,3,0.0,5,4,74.0,0.0,0.0,5,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,4,0.0,1.0,0,102.0,0.0,0.0,0.0,0.00,0,0.0,0.0,1.0,0,0.0,0,0,99.0,0.0,4,74.0,0,...,1,1,0,0,0,1,0,1,1,10,0.2222,0.15690,0.00000,0.4510,1.000,400.0,99.7,0,0,0,0,83574.5,27224.5,72116.5,81256.5,93381.5,71304.0,71132.5,71982.0,71643.5,123720.5,99329.5,124531.5,98782.5,94324.5,91465.0,73512.5,71757.5,120512.0,124972.5
216008,0,0,0.0,10,6,0.0,0.0,0.0,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0,0.0,0.0,0,106.0,0.0,0.0,0.0,0.00,0,0.0,0.0,0.0,0,0.0,0,1,93.0,0.0,0,0.0,0,...,0,0,0,0,0,0,0,0,0,7,0.2222,0.16980,0.00000,0.7734,0.000,360.0,97.7,0,0,0,3,83574.5,81598.5,72116.5,81256.5,93381.5,71304.0,71132.5,71982.0,71643.5,50996.5,117582.0,12565.5,98782.5,55646.0,9663.0,73512.5,71757.5,83594.0,68275.5
269221,0,3,84.0,0,5,0.0,84.0,0.0,5,0,0.0,0.0,0.0,0.0,84.0,0.0,59.7,2,4,0.0,0.0,2,85.0,0.0,0.0,11.5,7.26,0,0.0,0.0,0.0,0,0.0,0,1,92.0,0.0,0,0.0,0,...,0,0,0,0,0,0,1,1,0,12,0.1380,0.09410,0.00000,1.0700,1.131,192.0,97.3,0,0,0,6,83574.5,128109.5,72116.5,81256.5,23186.0,71304.0,71132.5,71982.0,71643.5,59988.5,27492.5,67044.0,98782.5,94324.5,36690.5,73512.5,71757.5,13105.5,84044.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
194102,1,4,12

# ENSEMBLE MODEL

In [ ]:
def _load_stack_file(type = 'valid'):
    stored_fd_path = folder_path+'/WiDS2021/ensemble_outputs/stacking_'+type+'/*.csv'
    file_paths = glob.glob(stored_fd_path)
    for path in file_paths:
      try:
        file_name = path[(len(folder_path+'/WiDS2021/ensemble_outputs/stacking_') +2*len(type)+2):-4]
        cols = [(file_name,) if type == 'test' else (file_name, 'diabetes_mellitus')][0]
        data_i = pd.read_csv(path).set_index('encounter_id')
        data_i.columns = cols 

        try:
          data[file_name] = data_i[file_name]
        except:
          data = data_i
      except: print('error at: ', file_name)
    return data

df_valid_stacked = _load_stack_file(type = 'valid')
df_test_stacked = _load_stack_file(type = 'test')

print(df_valid_stacked.shape)
print(df_test_stacked.shape)

In [ ]:
stack_to_train, stack_to_valid = train_test_split( df_valid_stacked, train_size= 0.75, random_state= 68)
# List feature of stacked dataframe
cols_stacked_ft = [i for i in df_valid_stacked.columns if i != col_target]

### SVC

In [ ]:
# Kiểm tra khả năng phân loại của model 2nd_layer
model = svm.SVC(probability=True)
cv_auc = cross_val_score( model, df_valid_stacked[cols_stacked_ft], df_valid_stacked[col_target], 
                         cv = 7, scoring = 'roc_auc', verbose = 1, n_jobs = -1)

print(cv_auc)
print(np.mean(cv_auc))

# TURNING HYPER PARAMS

## LightGBM

In [ ]:

def bayes_parameter_opt_lgb(init_round=15, opt_round=25, n_folds=5, random_seed=6, n_estimators=10000, learning_rate=0.05, output_process=False):
    # parameters
    def lgb_eval(num_leaves, feature_fraction, bagging_fraction, lambda_l1,learning_rate, lambda_l2, min_split_gain, min_child_weight,scale_pos_weight ):
        params = {'boosting_type': 'goss', 'application':'binary', 'early_stopping_round':400, 'metric':'auc', 
                  'max_depth':-1, 'force_col_wise': True, 'num_iterations': 10000,
                  # 'device': 'gpu', 'gpu_platform_id': 0, 'gpu_device_id': 0
                  }
        params["num_leaves"] = int(round(num_leaves))
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        # params['max_depth'] = int(round(max_depth))
        params['lambda_l1'] = max(lambda_l1, 0)
        params['learning_rate'] = max(learning_rate, 0)
        params['lambda_l2'] = max(lambda_l2, 0)
        params['min_split_gain'] = min_split_gain
        params['min_child_weight'] = min_child_weight
        params['scale_pos_weight'] = scale_pos_weight
        # params['num_iterations'] = int(round(num_iterations))
        cv_result = lgb.cv(params, train_data, nfold=n_folds, seed=random_seed, stratified=True, verbose_eval =2000, metrics=['auc'])
        return max(cv_result['auc-mean'])
    # range 
    lgbBO = BayesianOptimization(lgb_eval, {'num_leaves': (24, 45),
                                            'feature_fraction': (0.1, 0.9),
                                            'bagging_fraction': (0.25, 1),
                                            # 'max_depth': (-1, 9.99),
                                            'lambda_l1': (0, 7),
                                            'learning_rate' :(0.001, 0.1),
                                            'lambda_l2': (0, 5),
                                            'min_split_gain': (0.001, 0.1),
                                            'min_child_weight': (5, 50),
                                            'scale_pos_weight': (0.25,5),
                                            # 'num_iterations' : (5000, 30000),
                                            }
                                 , random_state=0)
    # optimize
    lgbBO.maximize(init_points=init_round, n_iter=opt_round)
    
    # output optimization process
    if output_process==True: lgbBO.points_to_csv("bayes_opt_result.csv")
    
    # return best parameters
    return lgbBO#.res['max']['max_params']

opt_params = bayes_parameter_opt_lgb(init_round=5, opt_round=15, n_folds=3, random_seed=6, n_estimators=10000, learning_rate=0.05)

## XGBoost

In [ ]:
def XGB_CV( max_depth, gamma, min_child_weight, max_delta_step, subsample, colsample_bytree ):
    params = {# fixed params
              'booster' : 'gbtree', 'seed' : 1001,
              'eta' : 0.1,
              'objective' : 'binary:logistic',
              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'auc',
              'gpu_id' : 0,
              'tree_method' : 'gpu_hist',
              # to search params
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              }
    xgbc = xgb.cv( params, dtrain, num_boost_round = 20000,
                    stratified = True, nfold = 5,
                   verbose_eval = 1000,
                    early_stopping_rounds = 500,
                    metrics = 'auc', show_stdv = True )
    
    return xgbc['test-auc-mean'].iloc[-1]

XGB_BO = BayesianOptimization(XGB_CV, 
	{
     'max_depth': (2, 12),
     'gamma': (0.001, 10.0),
     'min_child_weight': (0, 20),
     'max_delta_step': (0, 10),
     'subsample': (0.4, 1.0),
     'colsample_bytree' :(0.4, 1.0)
    })

XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

# ERROR FIXING

In [9]:
file_name = 'lgbm_dart_ft2020'
ss_train = pd.read_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_train/train_'+file_name+'.csv')
ss_valid = pd.read_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_valid/valid_'+file_name+'.csv')
ss_test = pd.read_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_test/test_'+file_name+'.csv')

In [20]:
ss_valid.index = X_valid.index
ss_test.index = df_test.index

In [22]:

# save to csv
ss_train[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_train/train_'+file_name+'.csv', index = True)
ss_valid[['prediction', 'diabetes_mellitus']].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_valid/valid_'+file_name+'.csv', index = True)
ss_test[ 'diabetes_mellitus'].to_csv(folder_path +'/WiDS2021/ensemble_outputs/stacking_test/test_'+file_name+'.csv', index = True)